<center> <img src="img/UMD-Wordmark.jpg" alt="Alt text" width="40%"> </center>

### <center>Sustainability Center Training Course</center>

## Table of Contents

  - [Table of Contents](#Table-of-Contents)
- [Welcome](#Welcome)
  - [Introduction](#Introduction)
  - [Fundamentals of Digital Engineering](#Fundamentals-of-Digital-Engineering)
- [Modules](#Modules)
- [🏠 Home](../../welcomePage.ipynb)

# Welcome

<center> <img src="img/Animation2.gif" alt="Alt text" width="40%"> </center>

## Introduction
Digital engineering (DE) is a technology that can
- increase the visibility of what is most urgent and important,
- identify how changes that are introduced affect a whole system, as well as parts of it, and
- enable stakeholders of a system to retrieve timely information about the status of a product moving through the development lifecycle at any point in time.
 
**Digital Perspective:** DE was initially conceived as the methodology and implementation for designing and establishing digital products or systems.  

<center>
    <img src="img/19 2.jpg" alt="Alt text" width="500">
</center>


**System engineering (SE) perspective:** How modern digital approaches facilitate the development of, from text-based systems engineering (document-centric) to model-based systems engineering (model-centric).

<center>
    <img src="img/20.jpg" alt="Alt text" width="600">
</center>

**Continues to evolve:** The varying colors of these keywords indicate shifts in thematic focus over time. Earlier approaches primarily concentrated on the use of digital technologies to address issues during the design phase, and keywords such as “CAD”, “CAE”, “FEM”, “design” were more frequently used in this phase. With the development of information technology, the dominant trends have naturally shifted towards more advanced digital technologies and systematic approaches. Recently, keywords like “digital twin”, “artificial intelligence”, “systems engineering”, and “model-based systems engineering” have become prevalent topics of discussion.

<center>
    <img src="img/21.jpg" alt="Alt text" width="900">
</center>

**Expected Benefits:** DE holds great expectations and benefits: (1) costs minimization; (2) global optimizations facilitation; and (3) prompt rapid responses to requirements.

<center>
    <img src="img/22.jpg" alt="Alt text" width="400">
</center>



Digital Engineering can help anyone in the entire enterprise, starting from managment and procurement to designers and engineers. Here is an example to show how Digital Engineering can be used by anyone

<center>
    <img src="img/26.jpg" alt="Alt text" width="500">
</center>

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
from PIL import Image
from fpdf import FPDF
from base64 import b64encode
import os
import io
import json  # PyMuPDF
# import myFunctions as myFn


# Create functions to be used later
def setYpos(currentSection, lastYpos, currentYpos, is_LR):
    lastYpos[is_LR] = currentYpos
    return lastYpos

def printPositionDebug(pos_debug, lastYpos, currentYpos, curretSection, sectionName, is_LR):
    LR_def = ["Left", "Right"]

    if pos_debug == 1:
        print("")
        print(f"🏃 Position Debug -- Section: {sectionName}")
        print(f"Current Section: {curretSection}")
        print(f"Current Side: {LR_def[is_LR]}")
        print(f"End Y Position: {currentYpos}")
        print(f"Last Y Position\n   Left: {lastYpos[0]}  |  Right: {lastYpos[1]}")
        print("")

def list_mbd_subfolders(parent_folder):
    """
    Lists all first-layer subfolders in the specified directory that have '.mbd' in their names.

    Args:
        parent_folder (str): Path to the parent folder.

    Returns:
        list: A list of subfolder paths with '.mbd' in their names.
    """
    if not os.path.isdir(parent_folder):
        print(f"The provided path '{parent_folder}' is not a valid directory.")
        return []

    # Get all first-layer subdirectories
    subfolders = [
        # os.path.join(parent_folder, subfolder)
        subfolder
        for subfolder in os.listdir(parent_folder)
        if os.path.isdir(os.path.join(parent_folder, subfolder)) and '.mbd' in subfolder
    ]

    return subfolders

# Set File Locations
MBD_Location = "example_resources/MBD_Files"
mbdFileList = list_mbd_subfolders(MBD_Location)

# Create a text box
text_box = widgets.Textarea(
    value='',
    placeholder='Type notes...',
    description='Notes:',
    disabled=False
)

# Variable to store the resulting text
resulting_text = ''

def update_text(change):
    global resulting_text
    resulting_text = change['new']
    # print(f"Updated Text: {resulting_text}")

# Attach the update function to the text box
text_box.observe(update_text, names='value')

# Create a dropdown menu
dropdown = widgets.Dropdown(
    options=mbdFileList,
    value=mbdFileList[0],
    description='MBD File:',
    disabled=False
)

# Variable to store the selected dropdown value
selected_option = mbdFileList[0]
def update_selection(change):
    global selected_option    
    selected_option = change['new']
    # print(f"Option selected: {selected_option}")

# Attach the update function to the dropdown
dropdown.observe(update_selection, names='value')

# Display the widget
display(dropdown, text_box)

In [ ]:
# A4 Dimentions
w = 210
h = 297

notes = resulting_text
outputFile = selected_option.replace(".mbd", ".pdf")

# partFile = "mBlock.mbd"
# partFile = "iceCleat.mbd"

partFile = MBD_Location + "/" + selected_option

# Import MBD Data
with open(f"{partFile}/info.json", "r") as file:
    partData = json.load(file)

# Setup Document Vairables
docFont = "dejavu-sans"
verticalSectionSpacing = 10
cellWidth = (w-20)/2-10
cellHeight = 8
smallSpacerHeight = 3

xPos = (10, (w-20)/2+20)
lastYpos = (0,0)

currentSection = 0

try:
    partName = partData["general_properties"]["name"]
    partID = partData["general_properties"]["part_id"]
except:
    print(f"🛑 Part Name or Part ID Missing 🛑")

header_start_page = 1
footer_start_page = 0

# Debug Enables
border_enable = False
pos_debug = False
column_choice_debug = False

output_real_file = False

# Setup FPDF Objects
class FPDF(FPDF):
    
    # Header
    def header(self):
        pageNum = self.page_no()
        if pageNum > header_start_page:
            self.set_font(docFont, "", 10)
            self.cell(0,5,f"{partID} [{partName}]", border=border_enable, align="R")
            self.image("example_resources/Images/UM-Dearborn_Logo-horizontal-RGB.png", h=5, x=10)
            self.set_xy(10,30)
            
    # Footer
    def footer(self):
        pageNum = self.page_no()
        footer_height = 5
        footer_font_size = 10

        if pageNum > footer_start_page:
            self.set_font(docFont, "", footer_font_size)
            self.set_y(-10-footer_height)
            self.cell((w-20)/3, footer_height,f"Page: {str(pageNum-footer_start_page)}", border=border_enable)
            self.cell((w-20)/3, footer_height,f"{partID} [{partName}]", border=border_enable, align="C")
            self.cell((w-20)/3, footer_height,"Data Sheet", border=border_enable, align="R")
    
    # Section Title
    def section_title(self, text, spacer_below_height):
        self.set_fill_color(0, 39, 76)
        self.set_font(docFont, "", 14)
        global is_LR

        # Determine if new cell should be left or right
        if lastYpos[0] == lastYpos[1]:
            self.set_xy(xPos[currentSection % 2], lastYpos[currentSection % 2])
            is_LR = currentSection % 2

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print(f'💈 Column Choice Debug -- Section: {text}')
                print(f'Columns are Identical in Length Chose: {currentSection % 2} out of {lastYpos}')
                print("")
        else:
            shortestColumn = lastYpos.index(min(lastYpos))
            self.set_xy(xPos[shortestColumn], lastYpos[shortestColumn])
            is_LR = shortestColumn

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print(f'💈 Column Choice Debug -- Section: {text}')
                print(f'Column {shortestColumn} is the shortest at {lastYpos[shortestColumn]} out of {lastYpos}')
                print("")
        
        # Set Font Settings for Section Title
        self.set_text_color(255, 202, 5)
        self.cell(cellWidth, cellHeight, "", border=border_enable, new_x="LEFT", new_y="NEXT")
        self.cell(cellWidth, cellHeight, text, fill=True, align="C", border=border_enable, new_x="LEFT", new_y="NEXT")
        
        # Spacer below section title
        self.cell(cellWidth, spacer_below_height, "", border=border_enable, new_x="LEFT", new_y="NEXT")
        
        # Reset Font for normal text
        self.set_text_color(0, 0, 0)
        self.set_font(docFont, "", 12)

    # Empty Section Title (For images and other non text based sections)
    def no_section_title(self):
        global is_LR
        self.set_fill_color(0, 39, 76)
        self.set_text_color(255, 202, 5)
        
        # Determine if new cell should be left or right
        if lastYpos[0] == lastYpos[1]:
            self.set_xy(xPos[currentSection % 2], lastYpos[currentSection % 2])
            is_LR = currentSection % 2

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print("💈 Column Choice Debug -- Titleless Section")
                print(f'Columns are Identical in Length Chose: {currentSection % 2} out of {lastYpos}')
                print("")
        else:
            shortestColumn = lastYpos.index(min(lastYpos))
            self.set_xy(xPos[shortestColumn], lastYpos[shortestColumn])
            is_LR = shortestColumn

            # Debug for column choice
            if column_choice_debug == True:
                print("")
                print("💈 Column Choice Debug -- Titleless Section")
                print(f'Column {shortestColumn} is the shortest at {lastYpos[shortestColumn]} out of {lastYpos}')
                print("")
            
        self.cell(cellWidth, cellHeight, "", border=border_enable, new_x="LEFT", new_y="NEXT") # Blank Cell

# Enable FPDF
pdf = FPDF("P", "mm", format=(w,h))

# Setup Fonts and Fallback Fonts
pdf.add_font(fname="example_resources/Fonts/TwitterColorEmoji-SVGinOT.ttf")
pdf.add_font(family="dejavu-sans", style="", fname="example_resources/Fonts/DejaVuSans.ttf")
pdf.add_font(family="dejavu-sans", style="b", fname="example_resources/Fonts/DejaVuSans-Bold.ttf")
pdf.set_fallback_fonts(["TwitterColorEmoji-SVGinOT", "dejavu-sans"])

pdf.set_font(docFont, "", 16)

## Page 1
pdf.add_page()

## Title
title_string = f"{partID} [{partName}]"
title_font_size = 25
pdf.set_text_color(0, 39, 76)
pdf.set_font(docFont, "b", title_font_size)

# Check to see how long title is and change font size accordingly
if pdf.get_string_width(title_string) <= 140:
    pdf.set_x(10)
elif pdf.get_string_width(title_string) >= (w-40):
    pdf.set_x(30)
    title_length = pdf.get_string_width(title_string)
    # Find new font size to make sure it will fit in title block
    while title_length >= (w-40):
        title_font_size += -1
        pdf.set_font(docFont, "b", title_font_size)
        title_length = pdf.get_string_width(title_string)
        if title_font_size <= 12.5:
            break
else:
    pdf.set_x(30)
    
pdf.multi_cell(0, 25/2, title_string, border=border_enable, align="C", max_line_height=title_font_size/2)

## Subtitle
pdf.set_font(docFont, "", 18)
pdf.set_y(21)
pdf.cell(0,9,"Data Sheet", border=border_enable, align="C", new_x="LEFT", new_y="NEXT")
# UMDearborn Logo
image = "example_resources/Images/UM-Dearborn_Logo-RGB.png"
pdf.image(image, h=20 , x=10, y=10)

lastYpos = [pdf.get_y(),pdf.get_y()]

pdf.set_text_color(0, 0, 0)
pdf.set_x(10)

## Sedctions

# Applications Secion
try:
    partApplications = partData["general_properties"]["applications"]
except:
    print("⚠️ general_properties: appliacations not found, skipping ⚠️")
else:
    pdf.section_title("Applications", smallSpacerHeight)

    text = ""

    for i in partApplications:
        if i == partApplications[0]:
            text = text + "  -  " + i
        else:
            text = text + "\n" + "  -  " + i  
        
    pdf.multi_cell(cellWidth, 6, text, border=border_enable, new_x="LEFT", new_y="NEXT")
    
    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Applications", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# Image
try:
    partImage = partData["general_properties"]["image"]
except:
    print("⚠️ general_properties: image not found, skipping ⚠️")
else:
    pdf.no_section_title()

    pdf.image(f"{partFile}/{partImage}", w=(w-20)/2-30, x = xPos[currentSection % 2]+10)

    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Image", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# Description
try:
    partDescription = partData["general_properties"]["description"]
except:
    print("⚠️ gneral_properties: description not found, skipping ⚠️")
else:
    pdf.section_title("Description", smallSpacerHeight)
    
    pdf.multi_cell(cellWidth, 5, partDescription, align="J", border=border_enable)

    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Description", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# Physical Properties
try:
    partPhysicalProperties = partData["physical_properties"]
except:
    print("⚠️ physical_properties not found, skipping ⚠️")
else:
    pdf.section_title("Physical Properties", smallSpacerHeight)

    # Material
    try:
        partMaterial = partPhysicalProperties["material"]
    except:
        print("⚠️ Part Material Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Material = {partMaterial}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Mass
    try:
        partMass = partPhysicalProperties["mass"]
        partMassUnit = partPhysicalProperties["mass_unit"]
    except:
        print("⚠️ Part Mass or Mass Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Mass = {partMass} [{partMassUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Volume
    try:
        partVolume = partPhysicalProperties["volume"]
        partVolumeUnit = partPhysicalProperties["volume_unit"]
    except:
        print("⚠️ Part Volume or Volume Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Volume = {partVolume} [{partVolumeUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Hardness
    try:
        partHardness = partPhysicalProperties["hardness"]
        partHardnessUnit = partPhysicalProperties["hardness_unit"]
    except:
        print("⚠️ Part Hardness or Hardness Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Hardness = {partHardness} [{partHardnessUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")
    
    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Physical Properties", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1


# Manufacturing Properties
try:
    partManufacturingProperties = partData["manufacturing_properties"]
except:
    print("⚠️ manufacturing_properties not found, skipping ⚠️")
else:
    pdf.section_title("Maufacturing Properties", smallSpacerHeight)

    # Method
    try:
        manufacturingMethod = partManufacturingProperties["method"]
    except:
        print("⚠️ Manufacturing Method Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Process = {manufacturingMethod}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Waste Percent
    try:
        manufacturingWaste = partManufacturingProperties["waste_percent"]
    except:
        print("⚠️ Manufacturing Waste Percent Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Waste = {manufacturingWaste}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Scrap Rate
    try:
        manufacturingScrap = partManufacturingProperties["scrap_rate"]
    except:
        print("⚠️ Manufacturing Scrap Rate Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Scrap = {manufacturingScrap}", border=border_enable, new_x="LEFT", new_y="NEXT")

    # Hardness
    try:
        manufacturingCarbonFootprint = partManufacturingProperties["carbon_footprint"]
        manufacturingCarbonFootprintUnit = partManufacturingProperties["carbon_footprint_unit"]
    except:
        print("⚠️ Carborn Footprint or Carbon Footprint Unit Missing ⚠️")
    else:
        pdf.cell(cellWidth, 5, f"Carbon Footprint = {manufacturingCarbonFootprint} [{manufacturingCarbonFootprintUnit}]", border=border_enable, new_x="LEFT", new_y="NEXT")
   
    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Maunfacturing Properties", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1


# Notes
try:
    notes = notes
except:
    print("😡 You messed with the code 😡\n😡 The notes are missing 😡")
else:
    pdf.section_title("Notes", smallSpacerHeight)
    pdf.multi_cell(cellWidth, 5, notes, border=border_enable, align="L")

    printPositionDebug(pos_debug, lastYpos, pdf.get_y(), currentSection, "Notes", is_LR)
    lastYpos = setYpos(currentSection, lastYpos, pdf.get_y(), is_LR)
    currentSection += 1

# # Add Second Page
# pdf.add_page()

# Output PDF file and write file to variable for display
pdf_bytes = pdf.output()
print(f"✅ PDF Generation Successful")

if output_real_file == True:
    pdf.output(outputFile)
    print(f"✅ Output Location: './{outputFile}'")

# New PDF Display
DisplayWidth, DisplayHeight = 800, 750
base64_pdf = b64encode(pdf_bytes).decode("utf-8")
display(HTML(f'<embed height="{DisplayHeight}" src="data:application/pdf;base64,{base64_pdf}" type="application/pdf" width="{DisplayWidth}"/>'))

# # Old PDF Display
# pdf_path = outputFile  # Replace with your PDF file path
# viewer = myFn.PDFViewer(pdf_path)
# viewer.show()

Here is another example of how another person can analize data while another person is generating a report

#### This example demonstrates how to run interactive modules on our JupyterHub platform. Press ▶️ to begin the interactive session and insert your name.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, FloatSlider, IntSlider
import ipywidgets as widgets
from IPython.display import display, FileLink
import os
from ipywidgets import fixed

print('Code Running...')
user_name = 'Unknown'

# Create a text input widget
name_input = widgets.Text(
    value='',
    placeholder='Type your name here',
    description='Insert Name',
    disabled=False
)

# Create an output widget
output = widgets.Output()

# Function to update the global variable and clear output
def update_name(change):
    global user_name
    user_name = change['new']
    with output:
        output.clear_output()  # Clear the previous output
        print(f"Name '{user_name}' will be provided to the following code cell.")

# Linking the text widget to the update function
name_input.observe(update_name, names='value')

# Display the text input widget and output widget
display(name_input, output)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider, FloatSlider
print('You can adjust the Line Degree of Complexity, and the Noise Level in Data Generated\n\nDrag Slider and Interact !')
# Initialize x data
x = np.linspace(-3, 3, 50)
np.random.seed(0)  # Set a fixed seed for reproducibility

# Function to generate data
def generate_data(noise, x=x):
    return x**2 + np.random.randn(50) * noise

# Cache the initial data
initial_noise = 0.8  # Initial noise level
y = generate_data(initial_noise)  # Initial data generation

# Function to plot polynomial regression
def plot_poly_regression(degree, y):
    # Reshape x for sklearn compatibility
    X = x[:, np.newaxis]
    
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)
    
    # Fit a linear model
    model = LinearRegression()
    model.fit(X_poly, y)
    
    # Predict and plot
    y_pred = model.predict(X_poly)
    plt.figure(figsize=(5, 5))
    
    
    # Adding the user's name as a transparent text in the background
    plt.text(0.5, 0.92, user_name + '\'s Plot', fontsize=22, color='#2F65A7', ha='center', va='center',
             alpha=1, transform=plt.gca().transAxes)  # Using axis transform for relative positioning
    plt.scatter(x, y, color='#006EB2', label='Data points')
    plt.plot(x, y_pred, color='#9A3324', label=f'Polynomial Degree {degree}', linewidth=2)
    plt.ylim([-40 - 10, 40 + 10])
    # plt.title(f"Polynomial Degree {degree} Fit with Noise Level {initial_noise}")
    # plt.legend()
    plt.grid(True)
    plt.show()

# Widgets for degree and noise adjustment
degree_slider = IntSlider(value=1, min=1, max=15, step=1, description="Degree")
noise_slider = FloatSlider(value=initial_noise, min=0.1, max=10, step=0.1, description="Noise Level")

# Track the previous noise level to decide whether to regenerate data
previous_noise = initial_noise

def interactive_update(degree, noise):
    global y, previous_noise
    if noise != previous_noise:
        y = generate_data(noise)  # Regenerate data if noise level has changed
        previous_noise = noise
    plot_poly_regression(degree, y)

# Interactive widget setup
interact(interactive_update,
         degree=degree_slider,
         noise=noise_slider);


## Fundamentals of Digital Engineering
Digital engineering is an integrated digital approach that uses authoritative sources of systems data and models as a continuum across disciplines to support lifecycle activities from concept through disposal. At its core, digital engineering evolves creating comprehensive digital (computer) models that represent every characteristic of a complex product or system that is to be developed. Originally designed for hardware-intensive lifecycle management, digital engineering also proves highly effective for software applications. Complex systems often depend on multiple subsystems operating simultaneously and in concert, known as a system of systems (SoS). This concept applies to both software and hardware-intensive systems. While integration challenges exist for both, the process for managing and scheduling hardware and software integration can differ significantly. Digital engineering addresses these challenges by providing a cohesive framework that ensures efficient and coordinated lifecycle management across all system components.

<center>
    <img src="img/01 2.jpg" alt="Alt text" width="900">
</center>

Digital engineering is sometimes referred to as “digital thread”, which encompasses a broader scope. Although the novelty of both terms has led to some debate, they are fundamentally extensions of product lifecycle management. This practice involves creating and storing a system's lifecycle artifacts in digital form, which can be updated as the system evolves. Both digital thread and digital engineering involve a single source of truth, referred to as the authoritative source of truth (ASoT). This ASoT contains all lifecycle artifacts in a centralized repository, ensuring that stakeholders work from the same models rather than multiple copies. The emerging processes of digital thread and digital engineering aim to address the difficulties of managing complex and evolving technologies over their lifecycles.

Digital engineering has an ambitious scope: managing the entire lifecycle of a system of systems, including both hardware systems and software components. Its emphasis on simulation and modeling offers significant benefits in hardware engineering alongside software engineering. By promoting the use of virtual models in tandem with, or prior to, constructing physical systems, digital engineering aims to improve efficiency, efficacy, and success. An offshoot of Model-Based Systems Engineering (MBSE), digital engineering seeks to accelerate the development of complex products, systems, and sub-systems. It achieves this by integrating various engineering disciplines and addressing each unique engineering challenge comprehensively. 

One application example of digital engineering is the digital twin, which is a software simulation that mirrors the operation of a physical system. These digital twins can also simulate software, reflecting the interdependence of physical and software components. Digital twins can be used to reduce the time, money, and power required to test and iterate on the design of novel technologies. In its current state, digital engineering is new and relatively unrecognized as an industry term.  The introduction of new standards and practices from digital engineering may come full-circle and influence the way that software is currently built, placing a larger emphasis on the modeling and simulation of software systems.

The objective of this training course is to familiarize the audience with the concepts and tools necessary for digital engineering, as well as the current challenges in the field. In contrast to traditional text/image-based course material, this course utilizes Jupyter notebooks to demonstrate the concepts and tools interactively. The format allows participants to practice these concepts at their own pace on an open-source platform. The course modules are designed to cater to a diverse audience (novices, learners and practitioners) accommodating various levels of experience and interest.

# Modules

### <font color = '#B8860B'>1. Yellow Belt Level</font>

- **[Module 1](Modules/1.%20Yellow%20Belt%20Level/Module1.ipynb) - Data** - Understand what data is and why it’s important in digital engineering.  
(*Introduction to basic data types (e.g., integer, string, Boolean), data collection methods, and basic data usage in engineering contexts.*)

- **[Module 2](Modules/1.%20Yellow%20Belt%20Level/Module2.ipynb) - Model Based Definition (MBD)** - Familiarize with model-based definitions and their purpose in engineering.  
(*Overview of what a model-based definition is, including basic terms and concepts like 3D CAD models, annotations, and tolerances.*)

- **[Module 3](Modules/1.%20Yellow%20Belt%20Level/Module3.ipynb) - Model Based Enterprise (MBE)** - Understand what MBE is and why it’s valuable.  
(*Introduction to MBE concepts and benefits, such as data integration, digital thread, and lifecycle management.*)

- **[Module 4](Modules/1.%20Yellow%20Belt%20Level/Module4.ipynb) - Modeling and Simulation** - Understand the basics of modeling and simulation.  
(*Overview of modeling types (e.g., static vs. dynamic models), simple simulations, and their purposes in engineering.*)

- **[Module 5](Modules/1.%20Yellow%20Belt%20Level/Module5.ipynb) - Model Based Process Control** - Understand how process data can be collected and used to improve performance.  
(*Introduction to process data collection methods, such as sensors.*)

- **[Module 6](Modules/1.%20Yellow%20Belt%20Level/Module6.ipynb) - Digital Twins** - Understand what a digital twin is and the concept’s basics.  
(*Define digital twins, common applications, and how they relate to physical assets in engineering.*)

- **[Module 7](Modules/1.%20Yellow%20Belt%20Level/Module7.ipynb) - Digital Engineering Practical Examples** - Identify the foundational elements of digital engineering.  
(*Introduction to digital engineering principles, such as CAD, PLM (Product Lifecycle Management), basic software used in the field, and the basics of process monitoring.*)

- **[Module 8](Modules/1.%20Yellow%20Belt%20Level/Module8.ipynb) - Digital Engineering Challenges** - Equip students with problem-solving techniques for basic digital engineering challenges.  
(*Address common issues in digital engineering, such as data compatibility and communication challenges.*)

### <font color = '#006400'>2. Green Belt Level</font>

- **[Module 1](Modules/2.%20Green%20Belt%20Level/Module1.ipynb) - Data** - Recognize and use different data types for engineering applications.  
(*Study data structures (e.g., lists, dictionaries), data formatting (CSV, JSON), and simple data analysis using tools like Excel or basic scripts.*)

- **[Module 2](Modules/2.%20Green%20Belt%20Level/Module2.ipynb) - Model Based Definition (MBD)** - Interact with and use MBD in simple engineering tasks.  
(*Hands-on experience with MBD systems, learning to interpret and use CAD annotations, dimensions, and simple assembly views for engineering components.*)

- **[Module 3](Modules/2.%20Green%20Belt%20Level/Module3.ipynb) - Model Based Enterprise (MBE)** - Learn how MBE integrates with various business practices.  
(*Exploration of MBE workflows in engineering, focusing on collaborative tools and MBE’s role in business operations like supply chain and quality management.*)

- **[Module 4](Modules/2.%20Green%20Belt%20Level/Module4.ipynb) - Modeling and Simulation** - Explore different modeling tools and perform basic simulations.  
(*Hands-on activities with common modeling tools, running simple simulations, and interpreting results for engineering decision-making.*)

- **[Module 5](Modules/2.%20Green%20Belt%20Level/Module5.ipynb) - Model Based Process Control** - Explore how process modeling can lead to process improvements.  
(*Use simple process models to analyze data trends, and introduce predictive maintenance concepts using basic data analysis tools.*)

- **[Module 6](Modules/2.%20Green%20Belt%20Level/Module6.ipynb) - Digital Twins** - Evaluate the utility of a digital twin in specific scenarios.  
(*Study and interact with basic digital twin examples, understanding how to create, monitor, and use them for simple engineering systems.*)

- **[Module 7](Modules/2.%20Green%20Belt%20Level/Module7.ipynb) - Digital Engineering Practice** - Apply digital engineering principles to solve practical problems.  
(*Learn about data management, version control, and workflow optimization using practical tools and simple software integration.*)

- **[Module 8](Modules/2.%20Green%20Belt%20Level/Module8.ipynb) - Digital Engineering Challenges** - Provide strategies to solve intermediate-level digital engineering problems.  
(*Troubleshoot digital engineering workflows, such as data sharing issues, software incompatibility, and basic cybersecurity concerns.*)

### <font color = '#000000'>3. Black Belt Level</font>

- **[Module 1](Modules/3.%20Black%20Belt%20Level/Module1.ipynb) - Data** - Apply data processing and analysis to solve engineering problems.  
(*Introduction to data processing and analysis using Python (pandas library, data cleaning, basic statistics) to extract insights from engineering datasets.*)

- **[Module 2](Modules/3.%20Black%20Belt%20Level/Module2.ipynb) - Model Based Definition (MBD)** - Integrate MBD into complex engineering workflows.  
(*Deep dive into advanced MBD functionalities, creating model-based definitions, and using MBD for detailed analysis and collaborative workflows.*)

- **[Module 3](Modules/3.%20Black%20Belt%20Level/Module3.ipynb) - Model Based Enterprise (MBE)** - Drive MBE adoption within complex organizational settings.  
(*Analyze case studies on MBE implementation, strategies for MBE integration in large enterprises, and potential business benefits such as reducing cycle times and improving data transparency.*)

- **[Module 4](Modules/3.%20Black%20Belt%20Level/Module4.ipynb) - Modeling and Simulation** - Perform complex simulations to drive engineering decisions.  
(*Advanced simulation techniques (e.g., finite element analysis), evaluating simulation results, and applying simulations in real-world engineering scenarios.*)

- **[Module 5](Modules/3.%20Black%20Belt%20Level/Module5.ipynb) - Model Based Process Control** - Design and optimize process control systems using advanced data models.  
(*Develop and implement process models to optimize performance, including advanced techniques like real-time monitoring, predictive analytics, and feedback loops.*)

- **[Module 6](Modules/3.%20Black%20Belt%20Level/Module6.ipynb) - Digital Twins** - Develop and deploy digital twins for complex systems.  
(*Explore advanced digital twin concepts, such as real-time data integration, IoT sensor data, and lifecycle management, and assess their impact on engineering projects.*)

- **[Module 7](Modules/3.%20Black%20Belt%20Level/Module7.ipynb) - Digital Engineering Practice** - Lead digital engineering practices in an organizational setting.  
(*Advanced use of digital engineering platforms, designing complex workflows, integrating cross-departmental digital engineering practices, and evaluating outcomes.*)

- **[Module 8](Modules/3.%20Black%20Belt%20Level/Module8.ipynb) - Digital Engineering Challenges** - Develop solutions for complex and emerging digital engineering challenges.  
(*Address advanced challenges like scalability, integration with legacy systems, and strategies for digital transformation, including managing data security and governance.*)
